First TF implementation

In [3]:
AEversion = '0'

In [4]:
import numpy as np
import os, sys, pickle
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
%autosave 60

Autosaving every 60 seconds


In [5]:
import gpustat
try:
    gpustat.print_gpustat()
except:
    pass

imperium-sm.hep.caltech.edu  Fri Jul 19 04:28:51 2019
[0] GeForce GTX 1080 | 22'C,   0 % |   466 /  8119 MB | ocerri(455M)
[1] GeForce GTX 1080 | 22'C,   0 % |    11 /  8119 MB |
[2] GeForce GTX 1080 | 25'C,   0 % |    10 /  8119 MB |
[3] GeForce GTX 1080 | 22'C,   0 % |    11 /  8119 MB |
[4] GeForce GTX 1080 | 25'C,   0 % |    11 /  8119 MB |
[5] GeForce GTX 1080 | 24'C,   0 % |    11 /  8119 MB |
[6] GeForce GTX 1080 | 24'C,   0 % |    11 /  8119 MB |
[7] GeForce GTX 1080 | 26'C,   0 % |   802 /  8119 MB | ocerri(791M)


In [6]:
import tensorflow as tf
from tensorflow.keras import layers

In [9]:
sys.path.append('../lib')
from progressBar import ProgressBar
from utils import EarlyStopping, createROC_curve, ELU_ProbNorm

In [7]:
dnd = []

## Get the dataset

In [10]:
from dataLoaders import ParticleDataset

In [11]:
dataset = ParticleDataset(template='../data/20190717_50part_PtOrder_v3/{}.npy', N_part=1, N_features=11)

In [12]:
dataset.loadTrainSM(N_train_max=1e6)
dataset.charge(dataset.SMMix_train)

Fetching Wlnu
Fetching qcd
Fetching Zll
Fetching ttbar
Expected 1.00M train
Expected 1.00M val

Loading Wlnu
Loading qcd
Loading Zll
Loading ttbar
+--------+----------+-------+------+
| Sample | Evts tot | Train | Val  |
+--------+----------+-------+------+
|  Wlnu  |  5618k   |  592k | 592k |
|  qcd   |  1166k   |  338k | 338k |
|  Zll   |  1777k   |  67k  | 67k  |
| ttbar  |  6542k   |   3k  |  3k  |
+--------+----------+-------+------+
Tot training 1.00 M
Tot val 1.00 M


In [13]:
dataset.loadValidationSamples('BSM', N_max=100000)

Loading Ato4l (47.7k)
Loading leptoquark (100.0k)
Loading hToTauTau (100.0k)
Loading hChToTauNu (100.0k)


## Define the model